# <center> <font style="color:rgb(100,109,254)"> Object Tracking In OpenCV with Tracker API  </font> </center>

We have already discussed some theory regarding different trackers that are available in the tracker module in OpenCV. In this notebook we'll see how to use them.

If you want to read more about each individual tracker then I would suggest you read their relevant papers linked below:

1. Tracking, learning and detection ([TLD](http://kahlan.eps.surrey.ac.uk/featurespace/tld/Publications/2011_tpami))
2. Multiple Instance Learning ([MIL](http://vision.ucsd.edu/~bbabenko/new/project_miltrack.shtml))
3. [CSRT]()
4. [BOOSTING](http://www.vision.ee.ethz.ch/boostingTrackers/Grabner2006RealTimeTracking.pdf)
5. [MEDIANFLOW](http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.231.4285&rep=rep1&type=pdf)
6. Kernelized Correlation Filters ([KCF](https://arxiv.org/pdf/1404.7584.pdf))
7.  Minimum Output Sum of Squared Error ([MOSSE](https://www.cs.colostate.edu/~vision/publications/bolme_cvpr10.pdf))
8. [GOTURN](http://davheld.github.io/GOTURN/GOTURN.pdf)

Its really simple, you can use all the trackers with the same code, all you need to change is just one line of code (The tracker creation line).

In [5]:
#Importing Required libraries
import cv2
import sys
import time

## <font style="color:rgb(134,19,348)">  Create the Tracker  </font>
You start by first Creating the tracker with the function 	`cv2.TrackerName_create()`, the `TrackerName` is the name of the tracker you want to use below we are creating the kCF tracker, if you want to use a different tracker you can use it using the table below:


**Syntax:**


|  **Tracker Names**         | **Function**    |
|---------------------------|-------|
| **BOOSTING**       | `cv2.TrackerKCF_create()`    |
| **MIL**        | `cv2.TrackerMIL_create()`     |
| **KCF**       | `cv2.TrackerKCF_create()`    |
| **TLD**     | `cv2.TrackerTLD_create()`    |
| **MEDIANFLOW** | `cv2.TrackerMEDIANFLOW_create()`   |
| **CSRT**  | `cv2.TrackerKCF_create()`    |
| **MOSSE**  | `cv2.TrackerMOSSE_create()`    |
| **GOTURN**           | `cv2.TrackerKCF_create()`    |


*See the tracker API [here](https://docs.opencv.org/trunk/d0/d0a/classcv_1_1Tracker.html)*

Since the usage of `GOTURN` is a deep learning model, it requires you to have its caffe model in your directory for tracking   and also it has some problems with OpenCV version 4.x, so we won't be using it in this notebook.

In [9]:
# Creating KCF tracker
tracker = cv2.TrackerKCF_create()

## <font style="color:rgb(134,19,348)"> Initialize Tracker  </font>
Now that we have created the tracker, lets initialize the tracker with the function `tracker.init()` , we will pass in the first frame of the video feed to be tracked and the location of the object to be tracked in that frame, the location can be either defined or better defined by the user by using `cv2.selectROI()`.

```Python
retval = cv2.Tracker.init(image, boundingBox)```

**Params:**

- **`image:`**	The initial frame
- **`boundingBox:`**	The initial bounding box


This function **returns** either `True` if initialization was successful, `False` otherwise.


In [10]:
# Read in the video on which we will perform tracking
cap = cv2.VideoCapture("Media/M4/american_pharoah.mp4")

# Read the first frame
ret, frame = cap.read() 

# Let the user select the object to be tracked
bounding_box = cv2.selectROI(frame,False)

# Then destroy the opened window.
cv2.destroyAllWindows()

# Initialize the tracker
status = tracker.init(frame, bounding_box)

print('Tracker Initialization Status: {}'.format(status))

Tracker Initialization Status: True


## <font style="color:rgb(134,19,348)">  Run the Tracker on the Video.  </font>

After initialization we can start Object tracking on the video, and then the tracker will continuously track our object by updating the tracker with every new frame by using a the function cv2.Tracker.update(), which also tells us if the tracker lost the object or not.

 ```Python
 retval, boundingBox = cv2.Tracker.update(image)```

- **`image`**	The current frame
- **`boundingBox`**	The bounding box that represent the new target location, if true was returned, not modified otherwise
- **`retval:`** True means that target was located and false means that tracker cannot locate target in current frame.

In [11]:
while True:

    ret, frame = cap.read()
    if not ret:
        break
 
    # Update the tracker
    status, bounding_box = tracker.update(frame)

    # Check if tracking was successful.
    if status:
        
        # Extract the x,y width and height from the bounding box.
        x, y, width, height = int(bounding_box[0]), int(bounding_box[1]), int(bounding_box[2]), int(bounding_box[3])
        cv2.rectangle(frame, (x,y), (x+width, y+height), (25,0,100), 2, 1)
        
    else :
        # In case tracker lost the target.
        cv2.putText(frame, "Tracker lost the target", (100,60), cv2.FONT_HERSHEY_SIMPLEX, 0.75,(0,0,255),2)

    # Showing the tracker type.
    cv2.putText(frame, "KCF Tracker", (100,20), cv2.FONT_HERSHEY_COMPLEX, 0.75, (50,170,50),2)
    
    # Show results
    cv2.imshow("Tracking", frame)

    # Exit if q pressed
    k = cv2.waitKey(10) 
    if k == ord('q'):
        break
        
cv2.destroyAllWindows()
cap.release()

## <font style="color:rgb(134,19,348)">   Tracking With All trackers  </font>

I've made a simple python script which will let you use different trackers on the same video by just pressing `a` on the keyboard. In this way you can check which tracker performs best on your sample video.

**Make a function that creates different trackers based on an index value.**

In [12]:
def selecttracker(frame, bounding_box, index):
    
    # set the scope of tracker variable to be global
    global tracker
    
    # Define the name of the available trackers in a list
    tracker_list = ['BOOSTING', 'TLD','KCF', 'MIL', 'CSRT', 'MEDIANFLOW', 'MOSSE']
    
    # Set the tracker name accoriding to the index
    tracker_name = tracker_list[index]
    
    # Create the specified tracker.
    if tracker_name == 'MIL':
        tracker = cv2.TrackerMIL_create()
        
    if tracker_name == "CSRT":
        tracker = cv2.TrackerCSRT_create()
        
    if tracker_name == 'TLD':
        tracker = cv2.TrackerTLD_create()
        
    if tracker_name == 'KCF':
        tracker = cv2.TrackerKCF_create()
        
    if tracker_name == 'MEDIANFLOW':
        tracker = cv2.TrackerMedianFlow_create()  
        
    if tracker_name == 'BOOSTING':
        tracker = cv2.TrackerBoosting_create()
        
    if tracker_name == 'MOSSE':
        tracker = cv2.TrackerMOSSE_create()
        
    status = tracker.init(frame, bounding_box)
    
    return tracker_name

**Now we can run the script below**

In [15]:
# Read sample video.
#cap = cv2.VideoCapture("Media/M4/american_pharoah.mp4")

# Instead you can choose your camera feed.
cap = cv2.VideoCapture(1)

# Read the first frame
ret, frame = cap.read() 

# Flipping the frame latterly
frame = cv2.flip(frame, 1) 

# We are starting with KCF tracker
index = 2

# Select the bounding box
bounding_box = cv2.selectROI(frame, False)  
cv2.destroyAllWindows()

# Initialize tracker with first frame and bounding box
tracker_name = selecttracker(frame, bounding_box, index)

# We are also going to record & display FPS, so setting fps to 0 initially
fps = 0

while True:
    
    # Note the starting time.
    start_time = time.time()
    
    ret, frame = cap.read()
    if not ret:
        break
        
    # Flip the frame 
    frame = cv2.flip( frame, 1 ) 
 
    # Update the tracker
    status, bounding_box = tracker.update(frame)

    # Check if tracking was successful.
    if status:
        
        # Extract the x,y width and height from the bounding box. 
        x, y, width, height = int(bounding_box[0]), int(bounding_box[1]), int(bounding_box[2]), int(bounding_box[3])
        cv2.rectangle(frame, (x,y), (x+width, y+height), (25,0,100), 2, 1)
        
    else :
        # In case tracker lost the target
        cv2.putText(frame, "Tracker lost the target", (100,60), cv2.FONT_HERSHEY_SIMPLEX, 0.75,(0,0,255),2)

    # Display tracker name
    cv2.putText(frame, "Type: {}".format(tracker_name), (100,20), cv2.FONT_HERSHEY_COMPLEX, 0.75, (50,170,50),2)
    
    # Show FPS 
    cv2.putText(frame, "FPS : {:.2f}".format(fps), (105,50), cv2.FONT_HERSHEY_COMPLEX, 0.77, (60,170,60), 2)
    
    # Show results
    cv2.imshow("Tracking", frame)
    
    # At the end subtract current time with start time and then divide 1 by this, this is your fps (Frames per second).
    fps = (1.0 / (time.time() - start_time))
 
    k = cv2.waitKey(1) 
    if k == 27 : 
        break
    
    # If the person presses `a` then change tracker type
    elif k == ord('a'):
        
        # Increment the index until the you've used up all the trackers,
        if index < 6:
           index += 1 
        else:
            index = 0            
            
        # Using the index create a new tracker 
        tracker_name = selecttracker(frame, bounding_box, index)
        
cv2.destroyAllWindows()
cap.release()